In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

# 전항지 출항 확인

전항지 출항 지점 확인

In [2]:
import os
 
path_dir = 'voy_path'
 
file_list = os.listdir(path_dir)
file_new_list = []
for i in file_list:
    if "portmis" in i:
        file_new_list.append(i)

In [3]:
# ATA 확인
def check_ATA(df_test):
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
            break
            continue
    return stop_index

# 시각화 툴로 processed된 것들 확인

항로 클러스터링

In [4]:
# dense_upper_china_list = ['COSCO_FOS_VRKR6_23processed'
# ,'AKA_BHUM_9V7282_7processed'
# , 'COSCO_FOS_VRKR6_25processed'
# , 'MSC_LA_SPEZIA_DIGX2_2processed'
# , 'OOCL_NEW_YORK_VRJS2_2processed'
# , 'MSC_WASHINGTON_5LDP4_4processed'
# , 'MSC_SOFIA_PAZ_D5FL9_2processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_26processed'
# , 'OOCL_HAMBURG_VRZK9_6processed'
# , 'SEAMAX_STRATFORD_V7EP2_6processed'
# , 'OOCL_LUXEMBOURG_VRGO3_5processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_25processed'
# , 'MSC_LUCIANA_3FTY3_4processed'
# , 'MSC_RENEE_VRPL2_5processed'
# , 'COSCO_FOS_VRKR6_24processed'
# , 'COSCO_FOS_VRKR6_20processed'
# , 'COSCO_FOS_VRKR6_26processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_23processed'
# , 'OOCL_HAMBURG_VRZK9_5processed']

# AKA_BHUM_voyages_list = [
#     'COSCO_FOS_VRKR6_2022_20processed', 'COSCO_FOS_VRKR6_2022_23processed',
#     'COSCO_FOS_VRKR6_2022_24processed', 'COSCO_FOS_VRKR6_2022_25processed',
#     'AKA_BHUM_9V7282_2022_7processed', 'COSCO_FOS_VRKR6_2022_27processed',
#     'COSCO_FOS_VRKR6_2022_28processed', 'AKA_BHUM_9V7282_2022_7processed',
#     'COSCO_FOS_VRKR6_2022_30processed'
# ]

AKA_BHUM_voyages_list = [
    'AKA_BHUM_9V7282_2022_7processed', 'AKA_BHUM_9V7282_2022_8processed'
]
OOCL_NEW_YORK_voyages_list = [
    'OOCL_NEW_YORK_VRJS2_2022_1processed',
    'OOCL_NEW_YORK_VRJS2_2022_2processed',
    'OOCL_NEW_YORK_VRJS2_2022_3processed'
]

OOCL_LUXEMBOURG_voyages_list = [
    'OOCL_LUXEMBOURG_VRGO3_2022_5processed',
    'OOCL_LUXEMBOURG_VRGO3_2022_6processed'
]

OOCL_GENOA_voyages_list = [
    'OOCL_GENOA_VROK9_2022_3processed', 'OOCL_GENOA_VROK9_2022_5processed'
]

OOCL_HAMBURG_voyages_list = [
    'OOCL_HAMBURG_VRZK9_2022_5processed', 'OOCL_HAMBURG_VRZK9_2022_6processed'
]

SEAMAX_STRATFORD_voyages_list = [
    'SEAMAX_STRATFORD_V7EP2_2022_6processed',
    'SEAMAX_STRATFORD_V7EP2_2022_7processed'
]

ZIM_CHARLESTORN_voyages_list = [
    'ZIM_CHARLESTON_VRUC3_2022_6processed',
    'ZIM_CHARLESTON_VRUC3_2022_7processed'
]

single_voyage_list = [
    'MSC_ARIANE_3FDA3_2022_1processed', 'MSC_DANIT_3FZU8_2022_2processed',
    'MSC_EVA_3EVM7_2022_1processed', 'MSC_EXPRESS_III_A8LG7_2022_1processed',
    'MSC_KANOKO_D5TM3_2023_1processed', 'MSC_LA_SPEZIA_DIGX2_2022_2processed',
    'MSC_VANESSA_HOWD_2023_1processed', 'MSC_VIRGO_CQEB9_2023_2processed',
    'MSC_SOFIA_PAZ_D5FL9_2022_2processed',
    'MSC_WASHINGTON_5LDP4_2022_4processed',
    'MSC_RAVENNA_A8ZU8_2022_3processed', 'MSC_VANDYA_3FDW4_2022_2processed',
    'MSC_RENEE_VRPL2_2022_5processed', 'SEASPAN_OCEANIA_VRBI2_2023_1processed',
    'SEASPAN_BREEZE_VRNL8_2023_1processed', 'VALENCE_9HA3398_2023_1processed'
]

In [5]:
upper_china_path_dir = 'voy_path/upper_china/processed/'

In [6]:
# 가는 길인데 굳이 오래 돌아갈 필요는 없잖아 알고리즘
def optimal_next_point(current_position, shortest_point_idx, shortest_distance,
                       path_df):
    posi_c = current_position
    latitude_i = path_df.loc[shortest_point_idx, "latitude"]
    longitude_i = path_df.loc[shortest_point_idx, "longitude"]
    posi_i = (latitude_i, longitude_i)
    min_d = shortest_distance
    n = shortest_point_idx + 1
    while True:
        try:
            latitude_n = path_df.loc[n, "latitude"]
            longitude_n = path_df.loc[n, "longitude"]
            posi_n = (latitude_n, longitude_n)
            distance = haversine(posi_c, posi_n, unit="km")
            if n == shortest_point_idx + 1:
                path_distance = haversine(posi_i, posi_n, unit="km")
                paths_distance = path_distance
            else:
                path_distance = haversine(posi_n, posi_previous_n, unit="km")
                paths_distance += path_distance

            if distance < (paths_distance + min_d):
                optimal_next_point_idx = n
                return optimal_next_point_idx
            elif n == path_df.index[-1]:
                print("no better optimal_point")
                return shortest_point_idx + 1
            else:
                n = n + 1
                posi_previous_n = posi_n
        except:
            print("no better optimal_point")
            return shortest_point_idx + 1

In [7]:
# 인덱스 번호를 가지고 항로의 segment 번호를 찾음
# 만약 segment의 마지막 번호라면 seg_no를 하나 더한다.

def find_segment_no(optimal_point_idx, path_df, the_number_of_segment):
    seg_no = optimal_point_idx // (len(path_df) / the_number_of_segment)
    next_no = (optimal_point_idx + 1) // (len(path_df) / the_number_of_segment)
    if seg_no != next_no:
        return int(next_no)
    else:
        return int(seg_no)

In [8]:
# df 내의 포인트 전체 길이 구하는 함수
def reckon_point_lengths(df_test_2):
    distance = 0
    df_test_2.reset_index(inplace=True, drop=True)
    for idx in df_test_2.index:
        if idx != 0:
            LONGITUDE = df_test_2.loc[idx, "longitude"]
            LATITUDE = df_test_2.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
            LATITUDE = df_test_2.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            distance += haversine(current_coord, previous_current_coord, unit="km")
    return distance

In [9]:
def cal_duration(speed, distance):
    hour = int(distance // km_h)
    minute = int(
        (distance / km_h - hour) * 60)
    second = ((distance / km_h - hour) * 60 - minute) * 60
    duration = datetime.timedelta(hours=hour,minutes=minute, seconds=second)
    return duration

# Tianjin to Busan PNIT

## AKA_BHUM

In [10]:
coordinates_array = pd.DataFrame(data={"latitude" : [],"longitude" : []})

for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    coordinates_array = coordinates_array.append(df[["latitude","longitude"]], ignore_index=True)

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
from shapely.geometry import LineString

In [13]:
from scipy import interpolate
import pandas as pd

## below 3 knot 제거

In [14]:
interpolation_number = "10000"

### interpolation 10000

In [15]:
# max_speed_list = []
# min_speed_list = []
# for i in AKA_BHUM_voyages_list:
#     df = pd.read_csv(upper_china_path_dir + i + ".csv")
#     data = df
#     min_speed = data["speed"].min()
#     max_speed = data["speed"].max()
#     max_speed_list.append(max_speed)
#     min_speed_list.append(min_speed)
# min_speed = np.min(min_speed_list)
# max_speed = np.max(max_speed_list)

# # norm = plt.Normalize(min_speed, max_speed)
# # cmap = plt.get_cmap('RdYlGn')
# # sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# # Create a ScalarMappable with the colormap and normalization

# mean_speed_dict = {"voyage" : [],"mean_speed" : []}

# for i in AKA_BHUM_voyages_list:
# #     fig, ax = plt.subplots()
    
#     df = pd.read_csv(upper_china_path_dir + i + ".csv")
#     data = df
#     # 3노트 이하 데이터 삭제
#     data = data[data["speed"] >= 3]
#     data.reset_index(inplace=True, drop=True)
#     min_speed = data["speed"].min()
#     max_speed = data["speed"].max()
#     vessel_name = data["vessel_name"].values[0]
#     voyage_name = i.split('processed')[0]
#     speed_std = data["speed"].std()
    
    
    
#     data['timestamp'] = pd.to_datetime(data['timestamp'])
#     data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

#     # Create a function that performs the interpolation
#     interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
#     interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
#     interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

#     # Specify the range of new x values for the interpolation
#     new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

#     # Use the interpolation function to estimate y values for the new x values
#     interp_longitude_y = interp_longitude(new_x)
#     interp_latitude_y = interp_latitude(new_x)
#     interp_speed_y = interp_speed(new_x)
#     # Create new DataFrame with the interpolated values
#     interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
# #     i = i.split("processed")[0]
#     interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
#     # plot the data
# #     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

# #     Add axis labels
# #     ax.set_xlabel('Longitude')
# #     ax.set_ylabel('Latitude')
# #     ax.set_title(f"{vessel_name}_{voyage_name}")
# #     # Add a colorbar
# #     cbar = plt.colorbar(sc,label='Speed')

# #         Show the plot
# #     plt.show()
        
#     check = data["speed"].apply(lambda x : True if x < 3 else False)
#     s = data.drop(data[check].index)["speed"]
#     print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
#     print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
#     print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
#     print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
#     print('-')
#     check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
#     s = interp_data.drop(interp_data[check].index)["speed"]
#     print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
#     print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
#     print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
#     mean_speed_dict['voyage'].append(voyage_name)
#     mean_speed_dict['mean_speed'].append(s.mean())
#     print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

In [16]:
# mean_speed_df = pd.DataFrame(mean_speed_dict)
# list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

In [17]:
# high_speed_path_data =[]
# low_speed_path_data =[]
# for i in AKA_BHUM_voyages_list:
#     i = i.split("processed")[0]
#     if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         low_speed_path_data.append(df)
# #     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
# #         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
# #         df = pd.read_csv(upper_china_path_dir + i + ".csv")
# #         high_speed_path_data.append(df)

In [18]:
# all_data = pd.concat(low_speed_path_data)

# # Add a new index column representing the row number
# all_data['index'] = range(len(all_data))
# all_data["index"] = all_data.index
# concaternated_df = all_data.groupby('index').mean()
# concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

In [19]:
result_df_dict=  {
    "experiment_no" : [],
    "mean_speed" : [],
    "max_speed" : []
}

# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [20]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [21]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 100

In [22]:
the_number_of_segment = 100

In [23]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [24]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [25]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

9.978291327149359
9.771655952384153
14.976630655037024
11.832478232092333
11.069202400757021
11.122228171245084
11.256532062161996
11.207134137925552
11.185461990422182
11.193346939030693
10.975830991526559
10.782755012456917
10.772611820963066
10.868706097856066
10.984669724853623
10.906564784263804
10.65792151945065
10.720041849702774
11.155884083218682
11.35003850419713
11.292582237410079
10.956164203494161
10.711341608955856
10.603474095291304
10.609806879443793
10.616139663596286
10.622472447748846
10.628924881294228
10.656939471017065
10.700875658039436
10.744811845061811
10.788748032083975
10.828561416075042
10.819382951769962
10.795973928234769
10.772564904699514
10.74915588116428
10.725746857629083
10.702337834093887
10.67892881055865
10.655519787023398
10.632110763488203
10.608701739953007
10.585292716417793
10.561883692882521
10.538474669347325
10.515065645812129
10.491656622276933
10.46824759874164
10.158344440963752
9.44295392574346
9.244892755702798
9.38768662869041
9.681

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [26]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [27]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}_voy_7".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [28]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:35:59.913062
평균: 0 days 01:06:07.031246089


In [29]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
264,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9472,34.2824,7.2,59.0,2022-11-05 01:41:07.975263,2022-11-04 16:40:24,2022-11-04 04:32:12,under way using engine,110.492280,204.631195,2022-11-05 10:36:09.096911,2022-11-05 10:36:09.691120,0 days 00:00:00.594209
259,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8914,34.2597,7.2,59.0,2022-11-05 01:15:53.638989,2022-11-04 16:15:15,2022-11-04 04:14:10,under way using engine,115.946719,214.732791,2022-11-05 10:37:47.787291,2022-11-05 10:36:09.691120,0 days 00:01:38.096171
268,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.0046,34.3002,7.2,69.0,2022-11-05 02:06:22.186771,2022-11-04 17:05:33,2022-11-04 04:56:12,under way using engine,105.347326,195.102764,2022-11-05 10:34:25.683778,2022-11-05 10:36:09.691120,0 days 00:01:44.007342
258,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8803,34.2556,7.2,62.0,2022-11-05 01:10:51.245788,2022-11-04 16:10:14,2022-11-04 03:32:10,under way using engine,117.003201,216.689391,2022-11-05 10:32:45.394090,2022-11-05 10:36:09.691120,0 days 00:03:24.297030
43,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.1595,38.6209,11.3,91.0,2022-11-02 22:57:05.351904,2022-11-02 13:53:43,2022-11-02 01:44:10,under way using engine,870.920106,1481.977755,2022-11-05 10:32:12.386940,2022-11-05 10:36:09.691120,0 days 00:03:57.304180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2264,35.3958,12.7,148.0,2022-11-04 03:48:49.927948,2022-11-03 18:48:26,2022-11-02 07:44:12,under way using engine,329.944606,586.228075,2022-11-05 08:15:18.424116,2022-11-05 10:36:09.691120,0 days 02:20:51.267004
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-05 08:10:52.238508,2022-11-05 10:36:09.691120,0 days 02:25:17.452612
24,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.5380,38.6838,12.0,101.0,2022-11-02 20:25:39.609523,2022-11-02 11:18:23,2022-11-02 08:14:16,under way using engine,922.954859,1599.735642,2022-11-05 08:00:46.644559,2022-11-05 10:36:09.691120,0 days 02:35:23.046561
83,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,122.7751,38.0019,11.1,97.0,2022-11-03 12:14:32.908857,2022-11-03 02:58:16,2022-11-02 07:44:12,under way using engine,636.722525,1079.193658,2022-11-05 13:11:47.569304,2022-11-05 10:36:09.691120,0 days 02:35:37.878184


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [30]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [31]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}_voy_7".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [32]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:35:59.913062
평균: 0 days 00:51:45.501616355


In [33]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
264,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9472,34.2824,7.2,59.0,2022-11-05 01:41:07.975263,2022-11-04 16:40:24,2022-11-04 04:32:12,under way using engine,110.492280,204.631195,2022-11-05 10:36:09.096911,2022-11-05 10:36:09.691120,0 days 00:00:00.594209
220,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.4674,34.1327,6.4,63.0,2022-11-04 21:59:03.571546,2022-11-04 12:58:23,2022-11-02 07:44:12,under way using engine,154.892014,307.900210,2022-11-05 10:35:56.300341,2022-11-05 10:36:09.691120,0 days 00:00:13.390779
178,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0116,34.0753,6.5,79.0,2022-11-04 18:27:03.963886,2022-11-04 09:26:14,2022-11-02 07:44:12,under way using engine,193.178862,380.144349,2022-11-05 10:36:24.562958,2022-11-05 10:36:09.691120,0 days 00:00:14.871838
237,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6501,34.1865,6.9,68.0,2022-11-04 23:24:53.599991,2022-11-04 14:24:23,2022-11-04 01:38:11,under way using engine,137.996749,261.750760,2022-11-05 10:35:00.105028,2022-11-05 10:36:09.691120,0 days 00:01:09.586092
197,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.2214,34.0942,6.4,80.0,2022-11-04 20:02:57.948151,2022-11-04 11:02:45,2022-11-02 07:44:12,under way using engine,175.792185,349.446361,2022-11-05 10:37:26.149523,2022-11-05 10:36:09.691120,0 days 00:01:16.458403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2264,35.3958,12.7,148.0,2022-11-04 03:48:49.927948,2022-11-03 18:48:26,2022-11-02 07:44:12,under way using engine,329.944606,586.228075,2022-11-05 08:15:18.424116,2022-11-05 10:36:09.691120,0 days 02:20:51.267004
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-05 08:10:52.238508,2022-11-05 10:36:09.691120,0 days 02:25:17.452612
24,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.5380,38.6838,12.0,101.0,2022-11-02 20:25:39.609523,2022-11-02 11:18:23,2022-11-02 08:14:16,under way using engine,922.954859,1599.735642,2022-11-05 08:00:46.644559,2022-11-05 10:36:09.691120,0 days 02:35:23.046561
83,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,122.7751,38.0019,11.1,97.0,2022-11-03 12:14:32.908857,2022-11-03 02:58:16,2022-11-02 07:44:12,under way using engine,636.722525,1079.193658,2022-11-05 13:11:47.569304,2022-11-05 10:36:09.691120,0 days 02:35:37.878184


# ETA 실험 (interpolation 10000), AKA BHUM 8항차 저속

In [34]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [35]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 100

In [36]:
the_number_of_segment = 100

In [37]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [38]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [39]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

9.978291327149359
9.771655952384153
14.976630655037024
11.832478232092333
11.069202400757021
11.122228171245084
11.256532062161996
11.207134137925552
11.185461990422182
11.193346939030693
10.975830991526559
10.782755012456917
10.772611820963066
10.868706097856066
10.984669724853623
10.906564784263804
10.65792151945065
10.720041849702774
11.155884083218682
11.35003850419713
11.292582237410079
10.956164203494161
10.711341608955856
10.603474095291304
10.609806879443793
10.616139663596286
10.622472447748846
10.628924881294228
10.656939471017065
10.700875658039436
10.744811845061811
10.788748032083975
10.828561416075042
10.819382951769962
10.795973928234769
10.772564904699514
10.74915588116428
10.725746857629083
10.702337834093887
10.67892881055865
10.655519787023398
10.632110763488203
10.608701739953007
10.585292716417793
10.561883692882521
10.538474669347325
10.515065645812129
10.491656622276933
10.46824759874164
10.158344440963752
9.44295392574346
9.244892755702798
9.38768662869041
9.681

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [40]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_8processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [41]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}_voy_8".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [42]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 11:37:49.205751
평균: 0 days 07:22:46.490695717


In [43]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
369,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.2604,34.3746,8.4,72.0,2022-12-26 23:35:55.676304,2022-12-26 14:35:39,2022-12-26 02:25:57,under way using engine,83.611932,6.031843,2022-12-27 05:20:27.139876,2022-12-27 05:21:48.345475,0 days 00:01:21.205599
364,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.2272,34.3263,7.9,10.0,2022-12-26 23:10:53.876834,2022-12-26 14:10:39,2022-12-26 01:49:59,under way using engine,89.731904,6.623141,2022-12-27 05:23:38.937825,2022-12-27 05:21:48.345475,0 days 00:01:50.592350
377,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.3649,34.4072,8.5,66.0,2022-12-27 00:16:02.467837,2022-12-26 15:15:48,2022-12-26 03:14:00,under way using engine,75.086391,5.396520,2022-12-27 05:19:27.782415,2022-12-27 05:21:48.345475,0 days 00:02:20.563060
378,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.3747,34.4106,8.6,66.0,2022-12-27 00:21:03.333108,2022-12-26 15:19:30,2022-12-26 03:14:00,under way using engine,74.277070,5.319768,2022-12-27 05:24:28.647686,2022-12-27 05:21:48.345475,0 days 00:02:40.302211
363,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.2247,34.3142,7.8,9.0,2022-12-26 23:05:50.693044,2022-12-26 14:05:09,2022-12-26 01:49:59,under way using engine,90.947079,6.749406,2022-12-27 05:18:35.754035,2022-12-27 05:21:48.345475,0 days 00:03:12.591440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2978,35.3333,8.5,150.0,2022-12-25 13:37:33.727335,2022-12-25 04:37:16,2022-12-24 05:43:59,under way using engine,322.703138,23.192937,2022-12-26 18:04:02.223503,2022-12-27 05:21:48.345475,0 days 11:17:46.121972
67,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2869,35.3439,9.5,149.0,2022-12-25 13:32:32.706657,2022-12-25 04:31:38,2022-12-24 05:43:59,under way using engine,323.814729,22.726682,2022-12-26 17:59:01.202825,2022-12-27 05:21:48.345475,0 days 11:22:47.142650
66,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2779,35.3535,9.7,150.0,2022-12-25 13:27:31.964447,2022-12-25 04:27:08,2022-12-24 05:43:59,under way using engine,324.745702,22.747130,2022-12-26 17:54:00.460615,2022-12-27 05:21:48.345475,0 days 11:27:47.884860
65,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2684,35.3641,9.8,150.0,2022-12-25 13:22:31.816395,2022-12-25 04:22:17,2022-12-24 05:43:59,under way using engine,325.737383,22.801407,2022-12-26 17:49:00.312563,2022-12-27 05:21:48.345475,0 days 11:32:48.032912


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [44]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_8processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [45]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}_voy_8".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [46]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 11:37:49.205751
평균: 0 days 06:36:57.912548517


In [47]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
369,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.2604,34.3746,8.4,72.0,2022-12-26 23:35:55.676304,2022-12-26 14:35:39,2022-12-26 02:25:57,under way using engine,83.611932,6.031843,2022-12-27 05:20:27.139876,2022-12-27 05:21:48.345475,0 days 00:01:21.205599
358,AKA BHUM,563135800,9307011.0,5888,KRBNP,12271201,128.2088,34.2879,0.6,69.0,2022-12-26 22:40:47.784087,2022-12-26 13:40:27,2022-12-26 12:43:59,under way using engine,94.167670,73.757404,2022-12-27 05:20:12.833699,2022-12-27 05:21:48.345475,0 days 00:01:35.511776
364,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.2272,34.3263,7.9,10.0,2022-12-26 23:10:53.876834,2022-12-26 14:10:39,2022-12-26 01:49:59,under way using engine,89.731904,6.623141,2022-12-27 05:23:38.937825,2022-12-27 05:21:48.345475,0 days 00:01:50.592350
377,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.3649,34.4072,8.5,66.0,2022-12-27 00:16:02.467837,2022-12-26 15:15:48,2022-12-26 03:14:00,under way using engine,75.086391,5.396520,2022-12-27 05:19:27.782415,2022-12-27 05:21:48.345475,0 days 00:02:20.563060
378,AKA BHUM,563135800,9307011.0,5888,KRBNP,12260900,128.3747,34.4106,8.6,66.0,2022-12-27 00:21:03.333108,2022-12-26 15:19:30,2022-12-26 03:14:00,under way using engine,74.277070,5.319768,2022-12-27 05:24:28.647686,2022-12-27 05:21:48.345475,0 days 00:02:40.302211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2978,35.3333,8.5,150.0,2022-12-25 13:37:33.727335,2022-12-25 04:37:16,2022-12-24 05:43:59,under way using engine,322.703138,23.192937,2022-12-26 18:04:02.223503,2022-12-27 05:21:48.345475,0 days 11:17:46.121972
67,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2869,35.3439,9.5,149.0,2022-12-25 13:32:32.706657,2022-12-25 04:31:38,2022-12-24 05:43:59,under way using engine,323.814729,22.726682,2022-12-26 17:59:01.202825,2022-12-27 05:21:48.345475,0 days 11:22:47.142650
66,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2779,35.3535,9.7,150.0,2022-12-25 13:27:31.964447,2022-12-25 04:27:08,2022-12-24 05:43:59,under way using engine,324.745702,22.747130,2022-12-26 17:54:00.460615,2022-12-27 05:21:48.345475,0 days 11:27:47.884860
65,AKA BHUM,563135800,9307011.0,5888,KRBNP,12261000,125.2684,35.3641,9.8,150.0,2022-12-25 13:22:31.816395,2022-12-25 04:22:17,2022-12-24 05:43:59,under way using engine,325.737383,22.801407,2022-12-26 17:49:00.312563,2022-12-27 05:21:48.345475,0 days 11:32:48.032912


# 결과 확인

In [48]:
result_pool = pd.DataFrame(result_df_dict)
result_pool

,experiment_no,mean_speed,max_speed
0,4_1_2_3_seg_100_voy_7,0 days 01:06:07.031246089,0 days 02:35:59.913062
1,4_2_1_3_seg_100_voy_7,0 days 00:51:45.501616355,0 days 02:35:59.913062
2,4_1_2_3_seg_100_voy_8,0 days 07:22:46.490695717,0 days 11:37:49.205751
3,4_2_1_3_seg_100_voy_8,0 days 06:36:57.912548517,0 days 11:37:49.205751


In [49]:
result_pool.sort_values(by="mean_speed")

,experiment_no,mean_speed,max_speed
1,4_2_1_3_seg_100_voy_7,0 days 00:51:45.501616355,0 days 02:35:59.913062
0,4_1_2_3_seg_100_voy_7,0 days 01:06:07.031246089,0 days 02:35:59.913062
3,4_2_1_3_seg_100_voy_8,0 days 06:36:57.912548517,0 days 11:37:49.205751
2,4_1_2_3_seg_100_voy_8,0 days 07:22:46.490695717,0 days 11:37:49.205751


In [51]:
for i in globals():
    if 'exp_result_df_' in i:
        globals()[i].to_csv(f"실험/230204_{i}", encoding="utf-8 sig", header=True, index=False)

In [ ]:
# exp_result_df_4_1_2_3

In [ ]:
# result_pool.to_csv(f"230126_result_pool_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)